In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

full_df = pd.read_csv('/content/drive/MyDrive/full_transactions.csv')

<ipython-input-2-76fa6f42fc2c>:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv('/content/drive/MyDrive/full_transactions.csv')


In [3]:
new_df = full_df.fillna('')

In [4]:
new_df['to'] = new_df['address to'] + new_df['contractInteracted']

In [5]:
new_df2 = new_df.drop(['address to','contractInteracted','Unnamed: 0'],axis=1)

In [6]:
new_df2.head()

,flag,address from,timestamp,amount,gasUsed,to
0,1,0xc6c198c7e472de4313f2ae9c9b02185dd8d91262,1594562749,10000000000000000,21000,0x000000000532b45f47779fce440748893b257865
1,1,0x028d61f42f1ca1ad6887f3311a0b1789ef6dadbb,1594563733,10000000000000000000,21000,0x000000000532b45f47779fce440748893b257865
2,1,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,1594563793,5000000000000000000,21000,0x000000000532b45f47779fce440748893b257865
3,1,0x2c2b7579e86363ccd46d0942f7fe274ccdbae351,1594563893,5000000000000000000,21000,0x000000000532b45f47779fce440748893b257865
4,1,0xabf1bbefc50800427ac69fa2e62ceac9b3b11d5b,1594564904,535414469040000000,21000,0x000000000532b45f47779fce440748893b257865


In [7]:
new_df3 = pd.concat([new_df2.iloc[0:25000,:],new_df2.iloc[25001:,:].sample(n=75000,random_state=0)])

In [8]:
# Create an empty graph
G = nx.MultiGraph()

# Add nodes to the graph for each unique address from , address to (might be contract too)
G.add_nodes_from(new_df3["address from"].unique(), type='addressFrom')
G.add_nodes_from(new_df3["to"].unique(), type='addressTo')

In [9]:
# Add edges and properties to the edges
for _, row in new_df3.iterrows():
    # Create a variable for each properties for each edge

        time = row["timestamp"],
        amt = row["amount"],
        gas = row["gasUsed"],


        G.add_edge(row['address from'], row['to'], time = time , amt = amt , gas = gas)

In [10]:
# Get the number of nodes and edges in the graph
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Print the number of nodes and edges
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 60013
Number of edges: 100000


In [11]:
adj_matrix = nx.adjacency_matrix(G).todense()

In [12]:
adj_matrix.shape

(60013, 60013)

In [13]:
# Prepare the data for input into the model
edge_list = list(G.edges(data=True))

In [14]:
class FraudGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FraudGNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(x)
        return x.squeeze(-1)

# Prepare the data for input into the model
x = []
for edge in edge_list:
    edge_values = list(edge[2].values())
    edge_values = [float(i[0]) if type(i) == tuple and type(i[0]) == str else i[0] if type(i) == tuple else i for i in edge_values]
    x.append(edge_values)
x = torch.tensor(x, dtype=torch.float)

In [15]:
target = torch.tensor(new_df3['flag'].values, dtype=torch.float)

In [16]:
# Define the model
input_dim = len(x[0])
hidden_dim = 16
model = FraudGNN(input_dim, hidden_dim)
num_epochs=201

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [17]:
# Train the model
for i in range(num_epochs):
    # Forward pass
    output = model(x)
    # Compute the loss
    loss = criterion(output, target)
    acc = (output.round() == target).float().mean()
    if i % 20 == 0:
        print(f'Epoch: {i}, Loss: {loss.item()}, Accuracy:{acc}')
    # Zero the gradients
    optimizer.zero_grad()
    # Perform backpropagation
    loss.backward()
    # Update the parameters
    optimizer.step()



Epoch: 0, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 20, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 40, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 60, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 80, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 100, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 120, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 140, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 160, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 180, Loss: 0.8426620960235596, Accuracy:0.7371699810028076
Epoch: 200, Loss: 0.8426620960235596, Accuracy:0.7371699810028076


In [18]:
new_df4 = new_df2.sample(n=100000,random_state = 0)

In [19]:
# Create an empty graph
G2 = nx.MultiGraph()

# Add nodes to the graph for each unique address from , address to (might be contract too)
G2.add_nodes_from(new_df4["address from"].unique(), type='addressFrom')
G2.add_nodes_from(new_df4["to"].unique(), type='addressTo')

In [20]:
# Add edges and properties to the edges
for _, row in new_df4.iterrows():
    # Create a variable for each properties for each edge

        time = row["timestamp"],
        amt = row["amount"],
        gas = row["gasUsed"],


        G2.add_edge(row['address from'], row['to'], time = time , amt = amt , gas = gas)

In [21]:
# Get the number of nodes and edges in the graph
num_nodes = G2.number_of_nodes()
num_edges = G2.number_of_edges()

# Print the number of nodes and edges
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 59534
Number of edges: 100000


In [22]:
adj_matrix2 = nx.adjacency_matrix(G2).todense()

In [23]:
# Prepare the data for input into the model
edge_list2 = list(G2.edges(data=True))

In [24]:
class FraudGNN2(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FraudGNN2, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x.squeeze(-1)

# Prepare the data for input into the model
x2 = []
for edge in edge_list2:
    edge_values = list(edge[2].values())
    edge_values = [float(i[0]) if type(i) == tuple and type(i[0]) == str else i[0] if type(i) == tuple else i for i in edge_values]
    x2.append(edge_values)
x2 = torch.tensor(x2, dtype=torch.float)

In [25]:
target2 = torch.tensor(new_df4['flag'].values, dtype=torch.float)

In [28]:
# Define the model
input_dim2 = len(x2[0])
hidden_dim2 = 16
model2 = FraudGNN2(input_dim2, hidden_dim2)
num_epochs=201

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.0001)

In [29]:
# Train the model
for i in range(num_epochs):
    # Forward pass
    output2 = model2(x2)
    # Compute the loss
    loss2 = criterion(output2, target2)
    acc = (output2.round() == target2).float().mean()
    if i % 20 == 0:
        print(f'Epoch: {i}, Loss: {loss.item()}, Accuracy:{acc}')
    # Zero the gradients
    optimizer.zero_grad()
    # Perform backpropagation
    loss2.backward()
    # Update the parameters
    optimizer.step()

Epoch: 0, Loss: 0.8426620960235596, Accuracy:0.9809200167655945
Epoch: 20, Loss: 0.8426620960235596, Accuracy:0.9809200167655945
Epoch: 40, Loss: 0.8426620960235596, Accuracy:0.9809200167655945
Epoch: 60, Loss: 0.8426620960235596, Accuracy:0.9809100031852722
Epoch: 80, Loss: 0.8426620960235596, Accuracy:0.9809100031852722
Epoch: 100, Loss: 0.8426620960235596, Accuracy:0.9809100031852722
Epoch: 120, Loss: 0.8426620960235596, Accuracy:0.9809100031852722
Epoch: 140, Loss: 0.8426620960235596, Accuracy:0.9809100031852722
Epoch: 160, Loss: 0.8426620960235596, Accuracy:0.9809100031852722
Epoch: 180, Loss: 0.8426620960235596, Accuracy:0.980400025844574
Epoch: 200, Loss: 0.8426620960235596, Accuracy:0.980400025844574
